In [1]:
import sys
sys.path.append('../..')

from utils import (load_data, CLASSIFICATION, REGRESSION, save_conf_matrix, 
                  TRAIN, TEST, MakiKFoldBalance, foldername2class)
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(1)
import matplotlib.pyplot as plt 
import sys
from glob import glob
from sklearn.utils import shuffle
import json
import copy


import tensorflow as tf
tf.config.optimizer.set_jit(False)

from tensorflow.python.platform import tf_logging as logging
from tensorflow.keras import Model
import tensorflow.keras.layers as L
from tensorflow.keras.utils import to_categorical
from classification_models.tfkeras import Classifiers
import tensorflow_addons as tfa

MODEL = 'resnet101'

USE_L1 = 'use_l1'
USE_L2 = 'use_l2'
USE_LABEL_SMOOTHING = 'use_label_smoothing'

L1_REG_15 = lambda: tf.keras.regularizers.l1(1e-5)
L2_REG_14 = lambda: tf.keras.regularizers.l2(1e-4)
L1_L2_REG_15_14 = lambda: tf.keras.regularizers.l1_l2(1e-5, 1e-4)
ALL_L1_L2 = [L2_REG_14]
ALL_L1 = [L1_REG_15]
ALL_L2 = [L1_L2_REG_15_14]
ALL_REG = ALL_L1_L2 + ALL_L1 + ALL_L2 + [None]

LABEL_SMOOTHING_VARS = [0.1]

class ExpConfig:
    reg: callable = None
    label_smoothing = None
    
    def to_dict(self) -> dict:
        return {
            'reg': self.reg().get_config() if self.reg is not None else self.reg,
            'label_smoothing': self.label_smoothing, 
        }
    
EPOCHS = 30 
BATCH_SIZE = 32
ITERATIONS_PER_EPOCH = 30_000 // BATCH_SIZE
NUM_K = 10
NUM_CLASSES = 21
PATH_DATASET = '../../double_modes'
SAVE_EXP_PATH = '/raid/rustam/danil/beams_loss_test'

In [2]:
@tf.function
def normalize_data(Xdata, Ydata):
    Xdata = tf.cast(Xdata, tf.float32)
    Xdata -= 128.0
    Xdata /= 128.0
    return Xdata, Ydata

@tf.function
def aug_noise(Xdata, Ydata):
    Xdata = tf.cast(Xdata, tf.float32)
    Xdata -= tf.random.normal(tf.shape(Xdata))
    return Xdata, Ydata

@tf.function
def aug_rotate(Xdata, Ydata, min_angle=0, max_angle=180):
    Xdata = tfa.image.rotate(
        Xdata,
        tf.random.uniform((tf.shape(Xdata)[0],), min_angle, max_angle),
        interpolation='nearest',
        fill_mode='reflect'
    )
    return Xdata, Ydata

@tf.function
def aug_cutout(Xdata, Ydata):
    mask = tf.ones_like(Xdata)
    size_cutout = tf.random.uniform([], 16, 32, dtype=tf.int32) // 2 * 2
    mask_cutout = tfa.image.random_cutout(mask, (size_cutout, size_cutout))
    noise_data = tf.ones_like(Xdata) * tf.random.normal(tf.shape(Xdata))
    Xdata = (1.0 - mask_cutout) * noise_data + Xdata * mask_cutout
    return Xdata, Ydata

@tf.function
def aug_brightness(Xdata, Ydata, parameter_bright=0.2):
    Xdata = tf.image.random_brightness(Xdata, parameter_bright)
    return Xdata, Ydata

@tf.function
def aug_contrast(Xdata, Ydata, min_cont=0.5, max_cont=2.0):
    Xdata = tf.image.random_contrast(Xdata, min_cont, max_cont)
    return Xdata, Ydata

@tf.function
def aug_saturation(Xdata, Ydata, min_saturation=0.75, max_saturation=1.25):
    Xdata = tf.image.random_contrast(Xdata, min_saturation, max_saturation)
    return Xdata, Ydata

def data_gen(Xdata, Ydata):
    counter = 0
    while True:
        yield Xdata[counter], Ydata[counter]
        
        counter += 1
        if counter == len(Xdata):
            counter = 0


def create_dataset(Xdata, Ydata_onehot, batch_size):
    tf_dataset = tf.data.Dataset.from_generator(
        lambda: data_gen(Xdata, Ydata_onehot),
        output_signature=(
            tf.TensorSpec(shape=(128, 128, 3), dtype=tf.uint8),
            tf.TensorSpec(shape=(NUM_CLASSES,), dtype=tf.int32),
        )
    )
    tf_dataset = tf_dataset.cache()
    tf_dataset = tf_dataset.shuffle(batch_size * 6, reshuffle_each_iteration=True)
    tf_dataset = tf_dataset.batch(batch_size)
    tf_dataset = tf_dataset.map(aug_noise, num_parallel_calls=tf.data.AUTOTUNE)
    tf_dataset = tf_dataset.map(normalize_data, num_parallel_calls=tf.data.AUTOTUNE)
    tf_dataset = tf_dataset.map(aug_rotate, num_parallel_calls=tf.data.AUTOTUNE)
    tf_dataset = tf_dataset.map(aug_cutout, num_parallel_calls=tf.data.AUTOTUNE)
    tf_dataset = tf_dataset.map(aug_brightness, num_parallel_calls=tf.data.AUTOTUNE)
    tf_dataset = tf_dataset.map(aug_contrast, num_parallel_calls=tf.data.AUTOTUNE)
    tf_dataset = tf_dataset.map(aug_saturation, num_parallel_calls=tf.data.AUTOTUNE)
    tf_dataset = tf_dataset.prefetch(4)
    return tf_dataset

In [3]:
class ConfMatrixCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, path_save: str, model: Model, Xtest: np.ndarray, Ytest: np.ndarray):
        self.path_save = os.path.join(path_save, 'conf_matrix')
        self.model = model
        self.Xtest = Xtest
        self.Ytest = Ytest
        
        os.makedirs(self.path_save, exist_ok=True)
    
    def on_epoch_end(self, epoch, logs=None):
        predictions = self.model.predict(self.Xtest)
        predictions = tf.nn.softmax(predictions, axis=-1)
        predictions = tf.argmax(predictions, axis=-1)
        predictions = predictions.numpy()
        path_save = os.path.join(self.path_save, str(epoch))
        save_conf_matrix(
            self.Ytest, predictions, 
            path_save,
            save_np=True
        )
        logging.info(f'Confusion matrix written to: {path_save}')

In [4]:
def single_folder_exp(exp_config: ExpConfig, path_to_exp: str, resnet_model_creation: Model, epochs=30, batch_size=32):
    # At start of each experiments - create new dataset (shuffle it)
    Xtrain, Ytrain, Xtest, Ytest, pred2params, config_data = load_data(
        path_exp_folder=path_to_exp,
        path_to_data=PATH_DATASET,
        use_saved=True,
        size_hw=(128, 128),
        test_percent=None,
        data_type=CLASSIFICATION,
        filename_config='config_data'
    )
    Ytrain = np.asarray(Ytrain).astype(np.int32)
    Ytest = np.asarray(Ytest).astype(np.int32)
    
    Ytrain_onehot = to_categorical(Ytrain)
    Ytest_onehot = to_categorical(Ytest)
    
    Xtest, _ = normalize_data(np.asarray(Xtest), None)
    
    train_dataset = create_dataset(Xtrain, Ytrain_onehot, BATCH_SIZE)
    
    # Create a MirroredStrategy.
    #strategy = tf.distribute.MirroredStrategy()
    #print("Number of devices: {}".format(strategy.num_replicas_in_sync))

    # Open a strategy scope.
    #with strategy.scope():
    model_backbone = resnet_model_creation(
        include_top=False, weights=None, 
        input_shape=(128, 128, 3)
    )

    x = L.GlobalAveragePooling2D()(model_backbone.output)
    x = L.Dropout(0.3)(x)
    x = L.Dense(21, use_bias=False)(x)

    model = Model(inputs=model_backbone.input, outputs=x)

    if exp_config.reg:
        for layer in model.layers:
            layer.kernel_regularizer = exp_config.reg()
            layer.bias_regularizer = exp_config.reg()
            layer.gamma_regularizer = exp_config.reg()
            layer.beta_regularizer = exp_config.reg()

    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(
            from_logits=True,
            label_smoothing=exp_config.label_smoothing
        ), 
        optimizer="adam", metrics=["accuracy"]
    )
    
    conf_matrix_callback = ConfMatrixCallback(path_to_exp, model, Xtest, Ytest)
    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        path_to_exp + '/checkpoints/{epoch:02d}',
        monitor='accuracy',
        verbose=0,
        save_best_only=False,
        save_weights_only=False,
        mode='auto',
        save_freq='epoch',
        options=None,
        initial_value_threshold=None,
    )
    print(f'Xtest={Xtest.shape}, Ytest={Ytest_onehot.shape}')
    history = model.fit(
        train_dataset, 
        batch_size=BATCH_SIZE, epochs=EPOCHS, 
        validation_data=(Xtest, Ytest_onehot), 
        callbacks=[checkpoint_callback, conf_matrix_callback],
        steps_per_epoch=ITERATIONS_PER_EPOCH
    )
    # Accuracy plot
    fig = plt.figure(figsize=(6,6))
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.legend(['accuracy', 'val_accuracy'])
    fig.savefig(os.path.join(path_to_exp, 'accuracy.png'))
    plt.close(fig)
    # Loss plot
    fig = plt.figure(figsize=(6,6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.legend(['loss', 'val_loss'])
    fig.savefig(os.path.join(path_to_exp, 'loss.png'))
    plt.close(fig)
    # Save loss info
    np.save(
        f'{path_to_exp}/loss.npy', 
        np.asarray(history.history['loss'], dtype=np.float32)
    )
    np.save(
        f'{path_to_exp}/val_loss.npy', 
        np.asarray(history.history['val_loss'], dtype=np.float32)
    )
    # Save accuracy info
    np.save(
        f'{path_to_exp}/accuracy.npy', 
        np.asarray(history.history['accuracy'], dtype=np.float32)
    )
    np.save(
        f'{path_to_exp}/val_accuracy.npy', 
        np.asarray(history.history['val_accuracy'], dtype=np.float32)
    )

In [5]:
def create_kfolds(path_to_exp_folder: str):
    # First - collect all data into massive
    data_path = []
    labels = []
    for single_folder in glob(os.path.join(PATH_DATASET, '*')):
        # Each folder - collect images
        folder_class = single_folder.split('/')[-1]
        for single_img_path in glob(os.path.join(single_folder, '*')):
            single_img_path = '/'.join(single_img_path.split('/')[-2:])
            data_path.append(single_img_path)
            labels.append(foldername2class[folder_class])
    # Second - shuffle dataset and create kfold obj
    data_path, labels = shuffle(data_path, labels)
    kf = MakiKFoldBalance(n_groups=NUM_K, shuffle_data=True)
    # Third - save each part as config file
    # i.e as json file with train and test data

    for inum_kfold, (train, test) in enumerate(kf.split(data_path, labels, NUM_CLASSES)):
        # Create folder with certain dataset config
        folder_exp_single = os.path.join(path_to_exp_folder, f'{inum_kfold}_exp')
        os.makedirs(folder_exp_single, exist_ok=True)
        config_data = {
            TRAIN: (np.asarray(copy.deepcopy(data_path))[train]).tolist(),
            TEST: (np.asarray(copy.deepcopy(data_path))[test]).tolist()
        }

        with open(os.path.join(folder_exp_single, 'config_data.json'), 'w') as fp:
            json.dump(config_data, fp)

In [6]:
EXP_NUMBER = 0

def model_exp(exp_config: ExpConfig):
    global EXP_NUMBER
    ResNetModel, preprocess_input = Classifiers.get(MODEL)
    exp_folder = f'{SAVE_EXP_PATH}/exp_{int(np.random.randint(0, 10_000_000))}'
    os.makedirs(exp_folder, exist_ok=True)
    with open(f'{exp_folder}/config_data.json', 'w') as fp:
        json.dump(exp_config.to_dict(), fp)
    
    create_kfolds(exp_folder)
    exp_folder_list = glob(os.path.join(exp_folder, '*_exp'))
    assert len(exp_folder) != 0
    for single_exp_folder in exp_folder_list:
        single_folder_exp(exp_config, single_exp_folder, ResNetModel, epochs=EPOCHS, batch_size=BATCH_SIZE)
        tf.keras.backend.clear_session()
    EXP_NUMBER += 1

In [ ]:
for reg in ALL_REG:
    for label_smoothing in LABEL_SMOOTHING_VARS: 
        try:
            exp_config = ExpConfig()
            if reg is not None:
                exp_config.reg = reg
            exp_config.label_smoothing = label_smoothing
            model_exp(exp_config)
        except Exception as e:
            with open(f'log_{EXP_NUMBER}.txt', 'a') as f:
                f.write(str(e))
                f.write(traceback.format_exc())

  3%|▎         | 27/945 [00:00<00:03, 269.29it/s]

Read data from config file:  config_data
Number of elements total: 21


100%|██████████| 105/105 [00:00<00:00, 263.49it/s]


train :  945
test:  105
Xtest=(105, 128, 128, 3), Ytest=(105, 21)
Epoch 1/30
937/937 [==============================] - ETA: 0s - loss: 2.1080 - accuracy: 0.3059INFO:tensorflow:Assets written to: /raid/rustam/danil/beams_loss_test/exp_9180544/6_exp/checkpoints/01/assets
INFO:tensorflow:Confusion matrix written to: /raid/rustam/danil/beams_loss_test/exp_9180544/6_exp/conf_matrix/0
937/937 [==============================] - 162s 156ms/step - loss: 2.1080 - accuracy: 0.3059 - val_loss: 2.5499 - val_accuracy: 0.2000
Epoch 2/30
937/937 [==============================] - ETA: 0s - loss: 1.4995 - accuracy: 0.5571INFO:tensorflow:Assets written to: /raid/rustam/danil/beams_loss_test/exp_9180544/6_exp/checkpoints/02/assets
INFO:tensorflow:Confusion matrix written to: /raid/rustam/danil/beams_loss_test/exp_9180544/6_exp/conf_matrix/1
937/937 [==============================] - 140s 149ms/step - loss: 1.4995 - accuracy: 0.5571 - val_loss: 1.6071 - val_accuracy: 0.6000
Epoch 3/30
937/937 [==========

Epoch 19/30
937/937 [==============================] - ETA: 0s - loss: 0.6963 - accuracy: 0.9579INFO:tensorflow:Assets written to: /raid/rustam/danil/beams_loss_test/exp_9180544/6_exp/checkpoints/19/assets
INFO:tensorflow:Confusion matrix written to: /raid/rustam/danil/beams_loss_test/exp_9180544/6_exp/conf_matrix/18
937/937 [==============================] - 140s 150ms/step - loss: 0.6963 - accuracy: 0.9579 - val_loss: 1.0487 - val_accuracy: 0.8000
Epoch 20/30
937/937 [==============================] - ETA: 0s - loss: 0.6995 - accuracy: 0.9560INFO:tensorflow:Assets written to: /raid/rustam/danil/beams_loss_test/exp_9180544/6_exp/checkpoints/20/assets
INFO:tensorflow:Confusion matrix written to: /raid/rustam/danil/beams_loss_test/exp_9180544/6_exp/conf_matrix/19
937/937 [==============================] - 140s 149ms/step - loss: 0.6995 - accuracy: 0.9560 - val_loss: 1.3200 - val_accuracy: 0.6857
Epoch 21/30
937/937 [==============================] - ETA: 0s - loss: 0.6885 - accuracy: 0.

...

Epoch 19/30
937/937 [==============================] - ETA: 0s - loss: 0.7039 - accuracy: 0.9531INFO:tensorflow:Assets written to: /raid/rustam/danil/beams_loss_test/exp_9180544/3_exp/checkpoints/19/assets
INFO:tensorflow:Confusion matrix written to: /raid/rustam/danil/beams_loss_test/exp_9180544/3_exp/conf_matrix/18
937/937 [==============================] - 139s 148ms/step - loss: 0.7039 - accuracy: 0.9531 - val_loss: 1.1521 - val_accuracy: 0.8286
Epoch 20/30
937/937 [==============================] - ETA: 0s - loss: 0.6996 - accuracy: 0.9545INFO:tensorflow:Assets written to: /raid/rustam/danil/beams_loss_test/exp_9180544/3_exp/checkpoints/20/assets
INFO:tensorflow:Confusion matrix written to: /raid/rustam/danil/beams_loss_test/exp_9180544/3_exp/conf_matrix/19
937/937 [==============================] - 146s 156ms/step - loss: 0.6996 - accuracy: 0.9545 - val_loss: 1.0384 - val_accuracy: 0.8762
Epoch 21/30
937/937 [==============================] - ETA: 0s - loss: 0.6988 - accuracy: 0.

  3%|▎         | 28/945 [00:00<00:03, 271.86it/s]

Read data from config file:  config_data
Number of elements total: 21


100%|██████████| 105/105 [00:00<00:00, 265.69it/s]


train :  945
test:  105
Xtest=(105, 128, 128, 3), Ytest=(105, 21)
Epoch 1/30
937/937 [==============================] - ETA: 0s - loss: 2.1197 - accuracy: 0.3012INFO:tensorflow:Assets written to: /raid/rustam/danil/beams_loss_test/exp_9180544/5_exp/checkpoints/01/assets
INFO:tensorflow:Confusion matrix written to: /raid/rustam/danil/beams_loss_test/exp_9180544/5_exp/conf_matrix/0
937/937 [==============================] - 163s 161ms/step - loss: 2.1197 - accuracy: 0.3012 - val_loss: 2.1050 - val_accuracy: 0.2857
Epoch 2/30
937/937 [==============================] - ETA: 0s - loss: 1.4891 - accuracy: 0.5648INFO:tensorflow:Assets written to: /raid/rustam/danil/beams_loss_test/exp_9180544/5_exp/checkpoints/02/assets
INFO:tensorflow:Confusion matrix written to: /raid/rustam/danil/beams_loss_test/exp_9180544/5_exp/conf_matrix/1
937/937 [==============================] - 145s 155ms/step - loss: 1.4891 - accuracy: 0.5648 - val_loss: 1.4246 - val_accuracy: 0.5238
Epoch 3/30
937/937 [==========

In [ ]:
ResNetModel, preprocess_input = Classifiers.get(MODEL)

In [ ]:
model_backbone = ResNetModel(
    include_top=False, weights=None, 
    input_shape=(128, 128, 3)
)

In [ ]:
layer = model_backbone.get_layer('relu0')
layer.kernel_regularizer = tf.keras.regularizers.l2(1e-4)
layer.bias_regularizer = tf.keras.regularizers.l2(1e-4)
layer.gamma_regularizer = tf.keras.regularizers.l2(1e-4)
layer.beta_regularizer = tf.keras.regularizers.l2(1e-4)

In [ ]:
layer.get_config()

In [ ]:
for layer in model_backbone.layers:
    layer.kernel_regularizer = tf.keras.regularizers.l2(1e-4)
    layer.bias_regularizer = tf.keras.regularizers.l2(1e-4)
    layer.gamma_regularizer = tf.keras.regularizers.l2(1e-4)
    layer.beta_regularizer = tf.keras.regularizers.l2(1e-4)

In [ ]:
model_backbone.summary()

In [ ]:
import numpy as np
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import tensorflow_addons as tfa

Xtrain = []
Ytrain = []

for _ in range(100):
    xxx = np.random.randn(128, 128, 3).astype(np.int64) * 0.0
    xxx[32:80, 32:80] = 255
    Xtrain.append(xxx)
    Ytrain.append(int(np.random.randint(0, 21)))
Xtrain = np.asarray(Xtrain).astype(np.float32)
Ytrain_onehot = to_categorical(np.asarray(Ytrain).astype(np.int32)).astype(np.float32)
batch_size=32

In [ ]:
ddd = next(iter(tf_dataset))

In [ ]:
plt.imshow(ddd[0][0])

In [ ]:
Xtrain, Ytrain, Xtest, Ytest, pred2params, config_data = load_data(
    path_exp_folder='/mnt/data/danil/beams_loss_test/exp_0/0_exp',
    path_to_data=PATH_DATASET,
    use_saved=True,
    size_hw=(128, 128),
    test_percent=None,
    data_type=CLASSIFICATION,
    filename_config='config_data'
)

In [ ]:
Ytrain_onehot = to_categorical(Ytrain)

In [ ]:
Ytest_onehot = to_categorical(Ytest)
Ytest_onehot.shape

In [ ]:
train_dataset = create_dataset(Xtrain, Ytrain_onehot, BATCH_SIZE)

In [ ]:
ddd = next(iter(train_dataset))

In [ ]:
Xtest, _ = normalize_data(np.asarray(Xtest), None)

In [ ]:
Xtest.shape

In [ ]:
plt.imshow((ddd[0][0] + 1)/2.0)